In [ ]:
%matplotlib inline
import os
import random
import numpy as np
import json
import matplotlib.pyplot
import cPickle as pickle
from matplotlib.pyplot import imshow
from PIL import Image
from sklearn.manifold import TSNE
from tqdm import tqdm
import rasterfairy

In [ ]:
my_data = pickle.load(open("./PATH/TO/SAVED/PICKLE.p", 'rb'))
print(len(my_data))

In [ ]:
#make sure that your image features matrix (e.g. pickle.dump) is pickled in 2.7 and not python3
#there's a variable 'protocol' here because i added the argument of specifying 2 when pickle dumping for python 2.7
images, pca_features, protocol = pickle.load(open('./PATH/TO/SAVED/PICKLE.p', 'rb'))

#wrap the objects you want to zip in a tuple vs the example in ml4a
for i, f in tuple(zip(images, pca_features))[0:5]:
    print("image: %s, features: %0.2f,%0.2f,%0.2f,%0.2f... "%(i, f[0], f[1], f[2], f[3]))

In [ ]:
#change this to the number of examples you have
num_images_to_plot = 100

if len(images) > num_images_to_plot:
    sort_order = sorted(random.sample(xrange(len(images)), num_images_to_plot))
    images = [images[i] for i in sort_order]
    pca_features = [pca_features[i] for i in sort_order]

In [ ]:
#experiment with different arguments, but ml4a gives a good note on the differences
X = np.array(pca_features)
tsne = TSNE(n_components=2, learning_rate=150, perplexity=25, angle=0.2, verbose=2).fit_transform(X)

In [ ]:
tx, ty = tsne[:,0], tsne[:,1]
tx = (tx-np.min(tx)) / (np.max(tx) - np.min(tx))
ty = (ty-np.min(ty)) / (np.max(ty) - np.min(ty))

In [ ]:
width = 4000
height = 3000
max_dim = 100

full_image = Image.new('RGBA', (width, height))
for img, x, y in tqdm(zip(images, tx, ty)):
    tile = Image.open(img)
    rs = max(1, tile.width/max_dim, tile.height/max_dim)
    tile = tile.resize((int(tile.width/rs), int(tile.height/rs)), Image.ANTIALIAS)
    full_image.paste(tile, (int((width-max_dim)*x), int((height-max_dim)*y)), mask=tile.convert('RGBA'))

matplotlib.pyplot.figure(figsize = (16,12))
imshow(full_image)

In [ ]:
# nx * ny = 1000, the number of images
# nx = 40
# ny = 25

nx = 10
ny = 10

# assign to grid
grid_assignment = rasterfairy.transformPointCloud2D(tsne, target=(nx, ny))

In [ ]:
#Did this to see what rasterfairy
print grid_assignment

In [ ]:
# tile_width = 72
# tile_height = 56

tile_width = 350
tile_height = 250

full_width = tile_width * nx
full_height = tile_height * ny
aspect_ratio = float(tile_width) / tile_height

grid_image = Image.new('RGB', (full_width, full_height))

#Be sure to specify that you want the first item inside grid_assignment
for img, grid_pos in tqdm(zip(images, grid_assignment[0])):
    idx_x, idx_y = grid_pos
    x, y = tile_width * idx_x, tile_height * idx_y
    tile = Image.open(img)
    tile_ar = float(tile.width) / tile.height  # center-crop the tile to match aspect_ratio
    if (tile_ar > aspect_ratio):
        margin = 0.5 * (tile.width - aspect_ratio * tile.height)
        tile = tile.crop((margin, 0, margin + aspect_ratio * tile.height, tile.height))
    else:
        margin = 0.5 * (tile.height - float(tile.width) / aspect_ratio)
        tile = tile.crop((0, margin, tile.width, margin + float(tile.width) / aspect_ratio))
    tile = tile.resize((tile_width, tile_height), Image.ANTIALIAS)
    grid_image.paste(tile, (int(x), int(y)))

matplotlib.pyplot.figure(figsize = (16,12))
imshow(grid_image)


In [ ]:
grid_image.save("./PATH/WHERE/YOU/WANT/TO/SAVE.jpg")